In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.signal
from utils import encode, ofdm, channels, audio, decode, preamble
import sounddevice as sd
cwd = os.getcwd()

In [2]:
DFT_LENGTH = 4096
CP_LENGTH = 512
OFDM_LENGTH = 4608
silence_duration = 1
chirp_duration = 1
fs = 48000

low_freq = 1000
high_freq = 10000
double_chirp = False
bits_per_symbol = 2

In [3]:
# Generate a bit array as data to transmit
new = True

spb = ofdm.subcarriers_per_block(fs,DFT_LENGTH,low_freq,high_freq)

if new:
    np.random.seed(8)
    info = np.random.randint(2, size=int(3.75 * 2 * spb * 128))
    np.savetxt('info', info, fmt='%i')

# Load a saved bit array
info = np.loadtxt('info', dtype=np.int32)
encoded = encode.ldpc_encode(info)

In [4]:
symbols = encode.qpsk_encode(encoded)

# shifted = ofdm.subcarrier_shift_gaussian(symbols, DFT_LENGTH, fs, low_freq, high_freq, 0.01, bits_per_symbol, constellation='qpsk')
shifted = ofdm.subcarrier_shift_ofdm(symbols, DFT_LENGTH, fs, low_freq, high_freq)
ofdm_data = ofdm.symbols_to_ofdm(shifted, DFT_LENGTH, CP_LENGTH)

In [5]:
#chirp between frame
chirp_inter = preamble.generate_chirp(chirp_duration, fs, low=low_freq, high=high_freq, silence_duration=0, double=double_chirp)

In [6]:
#generate known ofdm symbols
known_ofdm = True
repeat_time = 4

known_ofdm_data = preamble.load_known_ofdm(CP_LENGTH,repeat_time)

In [7]:
reshape_ofdm = np.reshape(ofdm_data,(-1,OFDM_LENGTH))

print(reshape_ofdm.shape)
i=0
frame_audio = np.array([])
while (i+128) < reshape_ofdm.shape[0]:
    ofdm_block = reshape_ofdm[i:i+128,:].flatten()
    frame = preamble.frame_assemble(chirp_inter,15*known_ofdm_data,15*ofdm_block)
    frame_audio = np.append(frame_audio,frame)
    i += 128

ofdm_block = reshape_ofdm[i:,:].flatten()
frame = preamble.frame_assemble(chirp_inter,15*known_ofdm_data,15*ofdm_block)
frame_audio = np.append(frame_audio,frame)

start_audio = preamble.transmission_start(fs,low_freq,high_freq,silence_duration)
end_audio = preamble.transmission_end(fs,low_freq,high_freq,silence_duration)

output_audio = np.concatenate((start_audio,frame_audio,end_audio))

(961, 4608)


In [8]:
#save and play
audio.signal_to_wav(np.real(output_audio), fs, 'new_tester_3.75_frames', 'audio_files')
# sd.play(output_audio, fs, blocking=True)